# DSBA 21/22 HSE & University of London

# Practical assignment 2. General DL, Outliers detection, Uncertainty estimation.

## General info
Release data: 31.10.2021

Soft deadline: 13.11.2021 23:59 MSK

Hard deadline: 16.11.2021 23:59 MSK

In this task, you are to build a NN for a binary classification task, calculate its uncertainty and find the most vague instances / outliers. We suggest using Google Colab for access to GPU. Competition link: https://www.kaggle.com/c/classification-task/

## Evaluation and fines

Each section has a defined "value" (in brackets near the section). Maximum grade for the task - 10 points, other points can be assigned to your tests.

Model training part can be shared among teammates, all the other parts must be done independently.

After the hard deadline is passed, the hometask is not accepted. If you send the hometask before the soft deadline, you **have a chance (if I'm free)** to get a feedback from the teacher and correct the errors.

Feel free to ask questions both the teacher and your mates, but __do not copy the code or do it together__ (refered to TracIn and Uncertainty Estimation part). "Similar" solutions are considered a plagiarism and all the involved students (the ones who gave & the ones who did) cannot get more than 0.01 points for the task. If you found a solution in some open source, you __must__ reference it in a special block at the end of your work (to exclude the suspicions in plagiarism).


## Format of handing over

The tasks are sent to the dropbox: https://www.dropbox.com/request/eKSHwbheVSJJdpvhdU2H. Don't forget to attach your name, surname & your group.


## 0. Data Download (0 points)

In [1]:
# Magic cell %%bash will run all the lines below in the terminal 

In [5]:
%%bash
kaggle competitions download -c classification-task
unzip -q classification-task.zip -d data/

100%|██████████| 454k/454k [00:00<00:00, 3.83MB/s]


In [3]:
import pandas as pd
train_df = pd.read_csv('data/train.csv')
train_df

,3,4,5,6,7,8,9,10,11,12,...,214,215,216,217,218,219,220,221,222,target
0,58.8,85.1,15.8,6.99,1.143700,0.928715,7.13,14.16,267.0,194.0,...,2.421053,2.739439,2.739439,2.739439,2.739439,0.473684,0.473684,0.473684,0.473684,True
1,44.8,71.1,23.4,6.84,0.954159,0.975350,9.99,7.66,191.0,287.0,...,2.631579,2.336756,2.336756,2.336756,2.336756,0.578947,0.578947,0.578947,0.578947,True
2,46.3,70.8,21.7,6.77,0.918434,1.118603,9.56,7.34,179.0,298.0,...,2.157895,2.120322,2.120322,2.120322,2.120322,0.368421,0.368421,0.368421,0.368421,False
3,50.2,77.5,24.4,6.87,1.037613,0.956836,9.60,9.53,195.0,239.0,...,2.368421,2.216415,2.216415,2.216415,2.216415,0.210526,0.210526,0.210526,0.210526,False
4,44.9,75.0,17.2,6.77,0.983691,0.948837,12.24,8.76,161.0,283.0,...,2.157895,2.604025,2.604025,2.604025,2.604025,0.421053,0.421053,0.421053,0.421053,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,55.1,82.8,17.8,6.87,1.098001,0.856129,7.95,12.75,293.0,203.0,...,2.666667,1.519160,3.464183,2.476621,2.486655,0.000000,0.000000,0.000000,0.000000,False
2276,49.0,78.0,23.1,6.68,0.819214,1.137980,10.17,10.38,203.0,215.0,...,3.666667,0.776903,4.154200,2.493900,2.475001,0.000000,1.000000,1.000000,0.666667,True
2277,46.4,76.5,20.8,6.68,0.899855,0.871232,13.00,7.99,225.0,281.0,...,2.666667,2.615180,1.439623,2.807280,2.287361,1.000000,0.000000,1.000000,0.666667,True
2278,57.9,85.2,18.7,6.86,0.905155,1.314188,9.01,14.66,326.0,169.0,...,1.333333,1.106967,0.797392,2.820050,1.574803,0.000000,0.000000,1.000000,0.333333,False


## 1. Model training (up to 7 points)

**Important!** Public Leaderboard contains only 50% of the test data. Your points will be measured wrt the whole test set, therefore your position on the LB after the end of the competition may change.

* test_accuracy > weak baseline (public LB): 2 points

* test_accuracy > strong baseline (public LB): + 1 point

* You are among 50% most successful students (private LB): + 2 point

* You are among 25% most successful students (private LB): + 2 point

## 2. TracIn (4 points)

In this section, you need to write a function for tracin score calculation to find out:

1) The most influential train instances (1 point)

2) Outliers among train instances (1 point)

Your main function must have the arguments, provided in the cell below - do not edit them (you may add additional args if needed)! It must output the TracIn score for each instance in the `dataloader` (in the same order). (For us to check the correctness of your solution)

In [14]:
def calculate_tracin_score(
    model: torch.nn.Module,
    weights_paths: List[str or Path],
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    work_dir: str or Path,
    nu_for_each_epoch: None
):
    if nu_for_each_epoch is None:
        nu_for_each_epoch = np.repeat(1, len(weights_paths))
    # Your code here ╰( ͡° ͜ʖ ͡° )つ──☆*:
    
    
    return scores

1) Find the most influential instances (the ones which have the biggest influence on the test points). Query their ids. (1 point)

2) Find the outliers and query their ids. (1 point)

In [14]:
# Your code here ╰( ͡° ͜ʖ ͡° )つ──☆*:

## 3. Uncertainty Estimation (2 points + 1 extra)

Use any of the methods for uncertainty estimation, demonstrated in the class, to estimate the uncertainty of your predictions (if you implement both MC dropout & Deep Ensemble, you'll get an extra point). Measure the uncertainty of the model on the test instances. Query the ids of the most uncertain ones.

In [11]:
# Your code here ╰( ͡° ͜ʖ ͡° )つ──☆*: